In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
using Gridap
using GridapGmsh
using Gridap.Geometry

In [ ]:
struct ThermistorInfo
    U::Float64
    k::NTuple{2,Float64}
    h::NTuple{2,Float64}
    Tw::Float64
    Tref::Float64
    material::NTuple{2,Symbol}
end

function ThermistorInfo(U=10.0; k=(1.0, 50.0), Dmm=(2.0, 0.5), Tw=200.0, Tr=0.0, Prₐ=0.707, kₐ=25.3e-3, νₐ=1.6e-5)
    
    D = Dmm .* 1e-3
    Re = (U/νₐ) .* D
    Nu = (0.51*Prₐ^0.37) .* Re .^ 0.5 
    
    h = kₐ .* Nu ./ D
    
    
    return ThermistorInfo(U, k, h, Tw, Tr, (:G, :S))
end

function matidx(m::Symbol) 
    if m==:G
        1
    else
        2
    end
end

velocity(th::ThermistorInfo) = th.U
temperature(th::ThermistorInfo) = th.Tw
hconv(th::ThermistorInfo, idx::Integer) = th.h[idx]    
kcond(th::ThermistorInfo, idx::Integer) = th.k[idx]    

hconv(th::ThermistorInfo, m::Symbol) = th.h[matidx(m)]    
kcond(th::ThermistorInfo, m::Symbol) = th.k[matidx(m)]    


In [ ]:
r(x) = x[2]
function thermistor(model, thermistor, fname; order=2)

    degree = 2*order
    reffe = ReferenceFE(lagrangian, Float64, order)
    
    Tw = thermistor.Tw
    Tref = thermistor.Tref
    
    V = TestFESpace(model, reffe, dirichlet_tags=["temp_glass", "temp_steel"])
    U = TrialFESpace(V, [Tw-Tref, Tw-Tref])

    Ω = Triangulation(model)
    dΩ = Measure(Ω, degree)    

    Γg = BoundaryTriangulation(model, tags="hglass")
    dΓg = Measure(Γg, degree)    
    
    Γs = BoundaryTriangulation(model, tags="hsteel")
    dΓs = Measure(Γs, degree)    
    
    Γtg = BoundaryTriangulation(model, tags="temp_glass")
    Γts = BoundaryTriangulation(model, tags="temp_steel")
    dΓts = Measure(Γts, degree)
    dΓtg = Measure(Γtg, degree)
    
    labels = get_face_labeling(model)
    dimension = 2
    tags = get_face_tag(labels, dimension);    
    tag_steel = get_tag_from_name(labels, "steel")
    tag_glass = get_tag_from_name(labels, "glass")    
    
    kg = kcond(thermistor, :G)
    ks = kcond(thermistor, :S)

    hg = hconv(thermistor, :G)
    hs = hconv(thermistor, :S)

    heatflux = (∇u, tag) -> begin
        if tag == tag_steel
            return ks * ∇u
        else
            return kg * ∇u
        end 
    end
    
    f(x) = 0.0

    a(u,v) = ∫( r*∇(v)⋅ (heatflux∘(∇(u), tags) ) )*dΩ  + ∫(r*v*u*hg)*dΓg + ∫(r*v*u*hs)*dΓs
    b(v) = ∫(r*v*f)*dΩ
    op = AffineFEOperator(a,b,U,V)
    
    ls = LUSolver()
    solver = LinearFESolver(ls)
    
    uh = solve(solver, op)
    
    writevtk(Ω, fname, cellfields=["T"=>uh+Tref])
    return uh, Ω, dΩ, Γs, dΓs, Γg, dΓg, Γts, dΓts, Γtg, dΓtg
end


In [ ]:
model = GmshDiscreteModel("mf58.msh")
th = ThermistorInfo(10)

In [ ]:
uh, Ω, dΩ, Γs, dΓs, Γg, dΓg, Γts, dΓts, Γtg, dΓtg = thermistor(model, th, "lixo")

In [ ]:
hg = hconv(th, :G)
hs = hconv(th, :S)
kg = kcond(th, :G)
ks = kcond(th, :S)
U = velocity(th)
Tw = temperature(th)


In [ ]:
Q̇g = 2 * 2π * hg * sum(∫(r*uh)*dΓg)
Q̇s = 2 * 2π * hs * sum(∫(r*uh)*dΓs)

Q̇g, Q̇s, Q̇g+Q̇s

In [ ]:
ng = get_normal_vector(Γg)
ns = get_normal_vector(Γs)

Q̇g1 = - 2 * 2π * kg * sum(∫(r*(∇(uh)⋅ng))*dΓg)

Q̇s1 = - 2 * 2π * ks * sum(∫(r*(∇(uh)⋅ns))*dΓs)

Q̇g1, Q̇s1, Q̇g1 + Q̇s1

In [ ]:
ntg = get_normal_vector(Γtg)
nts = get_normal_vector(Γts)

Q̇tg = 2 * 2π * kg * sum(∫(r*∇(uh)⋅ntg)*dΓtg)
Q̇ts = 2 * 2π * ks * sum(∫(r*∇(uh)⋅nts)*dΓts)

Q̇tg, Q̇ts, Q̇tg + Q̇ts

In [ ]:
Ag = 2π * sum(∫(r)*dΓg) * (10^3)^2
As = 2π * sum(∫(r)*dΓs) * (10^3)^2
Tg = 2π * sum(∫(r*uh)*dΓg) * (10^3)^2 / Ag
Ts = 2π * sum(∫(r*uh)*dΓg) * (10^3)^2 / As

In [ ]:
π*2*4 + 2π

In [ ]:
2 * (2π*1.0*1.5 + 2π * 0.75 * π/4  + π*(0.5^2-0.35^2)) + 2*π*0.35^2

In [ ]:
numstring(x, n=3) = string(x + 10^n)[2:end]

In [ ]:
model = GmshDiscreteModel("mf58.msh")

U = [0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0, 3.5, 4, 5, 6,7, 8, 10, 12, 14, 16, 18, 20]
fnames = "thermistor_" .* numstring.(round.(Int, 10*U))
th = ThermistorInfo.(U)
simul = thermistor.([model], th, fnames);

In [ ]:
function process_thermistor(data, th)
    uh, Ω, dΩ, Γs, dΓs, Γg, dΓg, Γts, dΓts, Γtg, dΓtg = data
    hg = hconv(th, :G)
    hs = hconv(th, :S)
    kg = kcond(th, :G)
    ks = kcond(th, :S)
    U = velocity(th)
    Tw = temperature(th)
    
    Q̇g = 2 * 2π * hg * sum(∫(r*uh)*dΓg)  # Heat flux from convection in glass surfaces
    Q̇s = 2 * 2π * hs * sum(∫(r*uh)*dΓs)  # Heat flux from convection in steel surfaces
    
    ng = get_normal_vector(Γg) # External normal to glass surfaces
    ns = get_normal_vector(Γs) # External normal to steel surfaces
    Q̇g1 = - 2 * 2π * kg * sum(∫(r*(∇(uh)⋅ng))*dΓg) # Heat flux from conduction in glass surfaces
    Q̇s1 = - 2 * 2π * ks * sum(∫(r*(∇(uh)⋅ns))*dΓs) # Heat flux from conduction in glass surfaces
    
    ntg = get_normal_vector(Γtg)
    nts = get_normal_vector(Γts)
    
    Q̇tg = 2 * 2π * kg * sum(∫(r*∇(uh)⋅ntg)*dΓtg)
    Q̇ts = 2 * 2π * ks * sum(∫(r*∇(uh)⋅nts)*dΓts)

    Ag = 2 * 2π * sum(∫(r)*dΓg)
    As = 2 * 2π * sum(∫(r)*dΓs)
    
    Tg = 2*2π * sum(∫(r*uh)*dΓg) / Ag
    Ts = 2*2π * sum(∫(r*uh)*dΓs) / As
    
    return U, Tw, Q̇g, Q̇s, Tg, Ts, Ag, As, Q̇g1, Q̇s1
end
    
    

In [ ]:
x = process_thermistor.(simul, th)
Q̇g = [y[3] for y in x]
Q̇s = [y[4] for y in x]
Q̇s1 = [y[10] for y in x]
Tg = [y[5] for y in x]
Ts = [y[6] for y in x];
Ag = x[1][7]
As = x[1][8]
W = Q̇g .+ Q̇s
Rw = 100.0
Ew = sqrt.(Rw .* W);

In [ ]:
hg = [x.h[1] for x in th]
W1 = hg .* Ag .* th[1].Tw
E1 = sqrt.(Rw .* W1);

In [ ]:
using CairoMakie
using CurveFit

In [ ]:
let fig = Figure()
    fit1 = PowerFit(E1, U)
    fitW = PowerFit(Ew, U)
    ax = Axis(fig[1,1])
    scatter!(ax, U, Ew, label="Real", color=:blue)
    lines!(ax, fitW.(Ew), Ew, color=:blue)
    scatter!(ax, U, E1, label="Ideal", color=:green)
    lines!(ax, fit1.(E1), E1, color=:green)
    axislegend(ax, position=:rb)
    fig
end


In [ ]:
power_fit(U, E1.^2)

In [ ]:
power_fit(U, Ew.^2)

In [ ]:
Q̇g

In [ ]:
scatterlines(U, Q̇g ./ Q̇s1)